In [1]:
!pip install crewai crewai_tools langchain langchain_community langchain_openai

Exception while exporting Span batch.
Traceback (most recent call last):
  File "C:\Users\250010891\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 464, in _make_request
    self._validate_conn(conn)
  File "C:\Users\250010891\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 1093, in _validate_conn
    conn.connect()
  File "C:\Users\250010891\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connection.py", line 741, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
  File "C:\Users\250010891\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connection.py", line 920, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
  File "C:\Users\250010891\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\util\ssl_.py", line 480, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(sock, context, tls_in_tls, server_hostnam

In [ ]:
from crewai import LLM
from langchain_ollama.llms import OllamaLLM

llm = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"
)

In [2]:
import os
from crewai import Agent, Task, Crew
from langchain_community.llms import Ollama
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

In [3]:
@tool
def ddgsearch(query: str):
    """
    Searches the web and returns results.
    """
    search_tool = DuckDuckGoSearchResults(num_results=10, verbose=True)
    return search_tool.run(query)

In [4]:
# Define Agents
researcher = Agent(
    role='Travel Researcher',
    goal='Gather detailed information about travel destinations including attractions, culture, and logistics.',
    backstory="""You are an expert travel researcher with a passion for uncovering unique destinations and cultural insights.
    You excel at finding reliable, up-to-date information to create comprehensive travel guides.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[ddgsearch]
)


In [5]:
writer = Agent(
    role='Travel Guide Writer',
    goal='Create engaging and informative travel guides based on research data.',
    backstory="""You are a skilled travel writer with a knack for crafting compelling narratives that inspire wanderlust.
    You transform raw research into polished, reader-friendly travel guides.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    #tools=[ddgsearch]
)

In [6]:
# Define Tasks
research_task = Task(
    description="""Conduct a comprehensive analysis of a specified travel destination (e.g., Paris, France).
    Include key attractions, cultural highlights, local cuisine, transportation options, and safety tips.
    Use the DuckDuckGo search tool to gather recent information.""",
    expected_output="A detailed report with sections for attractions, culture, cuisine, transportation, and safety.",
    agent=researcher
)

In [7]:

writing_task = Task(
    description="""Using the research report, write a concise and engaging travel guide for the destination.
    Structure the guide with an introduction, sections for attractions, culture, cuisine, transportation, and safety,
    and a conclusion encouraging travel. Keep the tone friendly and inspiring.""",
    expected_output="A polished travel guide in markdown format, approximately 500 words.",
    agent=writer
)

In [8]:
# Create and Run the Crew
#from crewai import Crew, Process
#crew = Crew(
#    agents=[researcher, writer],
#    tasks=[research_task, writing_task],
#    verbose=2,  # Detailed logging
#    process="sequential"
#)

In [9]:
from crewai import Crew, Process
crew = Crew(
agents=[researcher, writer],
tasks=[research_task, writing_task],
process=Process.sequential,
full_output=True,
share_crew=False,
verbose=True
)

In [10]:
# Execute the crew
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8ecf1b5e-5d00-4ccf-b1de-9b7f12041af4                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 44caf0f0-0e89-4eb3-85da-4e6642e4d7d9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 44caf0f0-0e89-4eb3-85da-4e6642e4d7d9
       Status: Executing Task...
    └── 🤖 Agent: Travel Researcher
            Status: In Progress

# Agent: Travel Researcher
## Task: Conduct a comprehensive analysis of a specified travel destination (e.g., Paris, France).
    Include key attractions, cultural highlights, local cuisine, transportation options, and safety tips.
    Use the DuckDuckGo search tool to gather recent information.


🤖 Agent: Travel Researcher
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Travel Guide Writer
## Task: Using the research report, write a concise and engaging travel guide for the destination.
    Structure the guide with an introduction, sections for attractions, culture, cuisine, transportation, and safety,
    and a conclusion encouraging travel. Keep the tone friendly and inspiring.


# Agent: Travel Guide Writer
## Final Answer: 
# Paris Travel Guide

**Introduction**
---------------

The City of Light, Paris, France - a destination that needs no introduction. With its stunning architecture, art museums, fashion boutiques, and romantic atmosphere, Paris is the perfect getaway for couples, solo travelers, and families alike. In this travel guide, we'll take you through the must-visit attractions, cultural experiences, culinary delights, transportation options, and safety tips to make your trip to Paris an unforgettable one.

**Attractions**
--------------

### Iconic Landmarks

*   **Eiffel Tower**: The iron lady of Paris is a must-visit attract

🚀 Crew: crew
├── 📋 Task: 44caf0f0-0e89-4eb3-85da-4e6642e4d7d9
│      Assigned to: Travel Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Travel Researcher
│           Status: ✅ Completed
└── 📋 Task: f00015be-7987-4cb1-966d-e7f6c4fad32a
       Status: Executing Task...
    └── 🤖 Agent: Travel Guide Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 44caf0f0-0e89-4eb3-85da-4e6642e4d7d9
│      Assigned to: Travel Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Travel Researcher
│           Status: ✅ Completed
└── 📋 Task: f00015be-7987-4cb1-966d-e7f6c4fad32a
       Assigned to: Travel Guide Writer
       Status: ✅ Completed
    └── 🤖 Agent: Travel Guide Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f00015be-7987-4cb1-966d-e7f6c4fad32a                                                                     │
│  Agent: Travel Guide Writer                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8ecf1b5e-5d00-4ccf-b1de-9b7f12041af4                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:

# Save the output to a file
with open("travel_guide2.md", "w") as f:
    f.write(str(result))

In [12]:
print("Travel guide generated and saved to travel_guide2.md")

Travel guide generated and saved to travel_guide2.md
